In [ ]:
import pandas as pd
import numpy as np
import os
import errno
import glob
import os.path
import shutil
import subprocess, shlex
import multiprocessing
from multiprocessing import Process
import time
import re
from sklearn.utils import shuffle

In [ ]:
#Hääle lugejate dataframe kirjutamine

#Kuna iga kõnekorpus on erinev, siis on raske luua universaalset lahendust
#Seega kui Teie kõnekorpuse ülesehitus erineb drastiliselt UT uudiskorpusest, siis peate meetodit muutma

#Tähtis on koodiploki lõpuks luua Dataframe data, kus on autosegmenteerimist soovivate failide nimekiri
#Vajalikud veerud on "speaker", kus on rääkija nimi, "filename", kus on helifaili asukoht ja "sent", kus on faili loetud lause


#Eeldame, et eksisteerib mingi juurkaust aadressil fileAddressBase, kus on iga kõneleja kohta kaust, kus asuvad csv failid, kus on kirjas helifailide asukohad ja loetud laused
#Samuti eeldame, et helifailid eksisteerivad mingis juurkaustas, kus on iga kõneleja kohta kaust, kus asuvad helifailid
#Samuti eeldame, et lause .txt failid eksisteerivad mingis juurkaustas, kus on iga kõneleja kohta kaust, kus asuvad lausete .txt failid (iga lause kohta 1 fail)

#NB! Lause all mõeldakse kogu helifailis etteloetud teksti. Kui helifailis loetakse ette mitu lauset, siis see loeb kui 1 lause.
UTspeakers = ['Mari', 'Kalev', 'Albert', 'Vesta'] #UT uudiste kõnekorpuse lugejad
EKIspeakers = ["Kylli","Meelis"] #EKI kõnekorpuse lugejad

fileAddressBase = '/home/ubuntu/JupyterFile/Sents/' #Juurkaust, kus asuvad lugejate kaustad, kus asuvad kõigi lausete csv failid
fileAddressPostfix = '/sentences.csv' #csv faili nimi, kus on kirjas eeltöötlust vajavate helifailide asukohtad ja sisseloetud laused

data = pd.DataFrame()

for speaker in UTspeakers:
    sentencesFile = fileAddressBase + speaker +  fileAddressPostfix
    sentences = pd.read_csv(sentencesFile, sep='|', header=None)
    sentences.columns = ['filename','sent']
    sentences['speaker'] = speaker
    
    sentences["filename"] = sentences["filename"].map(lambda filename: "UT-uudised-" + speaker + '/' + filename)

    
    data = data.append(sentences)

for speaker in EKIspeakers:
    sentencesFile = fileAddressBase + speaker +  fileAddressPostfix
    sentences = pd.read_csv(sentencesFile, sep='|', header=None)
    sentences.columns = ['filename', 'sent']
    sentences['speaker'] = speaker
    
    sentences["filename"] = sentences["filename"].map(lambda filename: "EKI-ilukirjandus-" + speaker + '/' + filename)
    
    data = data.append(sentences)

#Edasi askeldame koopiaga, nõnda saab notebook lõpus olevat andmevalikut paremini kasutada
combinedData = data.copy()

In [ ]:
#https://reactgo.com/python-run-mutliple-commands/
#Kasutusel autosegmenteerija rakendamisel, et luua TextGrid, kus on kirjas, millal on vaikus
    #Vajalik, kuna tahame kasutada Shelli ja teha seda lõimedega
def subprocess_cmd(command):
    process = subprocess.Popen(command,stdout=subprocess.PIPE,stderr=subprocess.PIPE, shell=True)
    ret = process.wait()
    proc_stdout = process.communicate()
    
#https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory
def ifDirNotExistCreate(outputPath):
    if not os.path.exists(os.path.dirname(outputPath)):
        try:
            os.makedirs(os.path.dirname(outputPath))
        except OSError as exc: # Võidusõidu vastane kaitse
            if exc.errno != errno.EEXIST:
                raise


In [ ]:
#Loome igale datas olevale andmereale vastava TextGrid faili kasutades autosegmenteerijat https://bark.phon.ioc.ee/autosegment2/,
    # milles on muuhulgas kirjas, mis hetkel algavad ja lõppevad vaikused

    
    # https://stackoverflow.com/questions/7207309/how-to-run-functions-in-parallel
    # https://testdriven.io/blog/developing-an-asynchronous-task-queue-in-python/
start = time.time() #Mõõdame töö aega

processes = 3 #Mida rohkem multithreadingut, seda suurem tõenäosus, et autosegmenteerijal tekib probleeme ja peab hiljem uuesti proovima
              #3 tundus olevat kiiruse mõttes optimaalne
proc = [] #Salvestame siia kuni processes arv lõime kausutavat protsessi
          #Teisisõnu saame korraga luua TextGride processes arv helifailile

#Jörgmised kaustad on juurkaustad, seega tavaliselt on neis alamkaustad, kus asuvad vajalikud failid
#Kaust kust loetakse helifailid sisse
audioFileBaseLoc = "."
#Kaust kust loetakse lausete .txt failid (iga lause eraldi txt fail)
sentenceFileBaseLoc = "SentencesWithPreprocessingTextFiles"
#Kaust kuhu kirjutatakse autosegmenteerija loodud TextGridid
outputFileBaseLoc = "TextGridsWithPreprocessedSents"

#Autosegmenteerija serveri aadress
autosegmentAddress = "http://10.15.0.2:8888/run"

#Võtab ühe töö järjekorrast ja laseb sellest TextGrid luua
def curlOneFileFromQueueToAutosegment(task_queue):
    while not task_queue.empty():
        if(task_queue.qsize() % 50 == 0):
            print("Files left to multithread originally:",task_queue.qsize())
        filename = task_queue.get()
        curlOneFileToAutosegment(filename)
    return True

#Loob vastava failinimega audiofailile TextGridi
def curlOneFileToAutosegment(filename):
    baseFilename = (".").join(filename.split(".")[0:-1])
    ifDirNotExistCreate(outputFileBaseLoc +"/"+baseFilename+'.TextGrid')
    subprocess_cmd('curl --fail  --insecure  -F "wav=@'+
                    audioFileBaseLoc + "/" + baseFilename+'.wav" -F "txt=@'+
                    sentenceFileBaseLoc + "/" + baseFilename+'.txt" '+ autosegmentAddress + ' > '+
                    outputFileBaseLoc +'/'+ baseFilename+'.TextGrid')
    

task_queue = multiprocessing.Queue()
listOfFileNames = combinedData["filename"].tolist() #Itereerime listi peal, seega konverteerime

#Paneme iga helifaili tööde järjekorda
for fileName in listOfFileNames:
    task_queue.put(fileName)
        
for n in range(processes): #Limiteerime lõimede arvu processes muutujaga
    p = Process(target=curlOneFileFromQueueToAutosegment, args=(task_queue,))
    proc.append(p)
    p.start()

for p in proc:
    p.join()
    
#Kuna eelmine TextGrid loomine ei tööta 10-20% juhtudel threading tõttu, käime kõik uuesti üle, ilma threadingutta
#Ebaõnnestumise korral on loodud TextGrid suuruseks 0, seega saab niimoodi kontrollida failid üle
#Kui TextGrid on tühi, siis proovime uuesti luua (Kui probleemiks oli mingi sümbol, siis loomulikult on ka siin tulemuseks tühi textgrid)

#Statistika kogumiseks

failedFileCounter = 0
totalFileCounter = 0
maxPossibleFileAmount = len(listOfFileNames)

for filename in listOfFileNames:
    baseFilename = (".").join(filename.split(".")[0:-1])
    if os.path.getsize(outputFileBaseLoc + '/'+ baseFilename+".TextGrid") == 0:
        curlOneFileToAutosegment(filename)
        failedFileCounter += 1
    totalFileCounter += 1
    if(totalFileCounter % 50 == 0):
        print("Have checked a total of",totalFileCounter, "files out of", maxPossibleFileAmount)
    
print("Fail rate = ",failedFileCounter/totalFileCounter)
print("Time taken = ",time.time() - start, "s")
print("Total files = ", totalFileCounter)

Files left to multithread originally: 50
Have checked a total of 50 files out of 60
Fail rate =  0.06666666666666667
Time taken =  94.31578540802002 s
Total files =  60


In [ ]:
#Kui on soov testimise eesmärgil valida vaid osa andmeid

# limiit = 10

# VestaData= (data[data["speaker"]=="Vesta"])
# VestaData= VestaData[0:limiit]

# AlbertData= (data[data["speaker"]=="Albert"])
# AlbertData= AlbertData[0:limiit]

# KalevData= (data[data["speaker"]=="Kalev"])
# KalevData= KalevData[0:limiit]

# MariData= (data[data["speaker"]=="Mari"])
# MariData= MariData[0:limiit]

# KylliData= (data[data["speaker"]=="Kylli"])
# KylliData= KylliData[0:limiit]

# MeelisData= (data[data["speaker"]=="Meelis"])
# MeelisData= MeelisData[0:limiit]


# combinedData = VestaData.append([AlbertData,KalevData,MariData,KylliData,MeelisData])

In [ ]:
#Kui on soov testimise eesmärgil valida vaid osa andmeid, suvaliselt

# limiit = 5

# VestaData= shuffle(data[data["speaker"]=="Vesta"])
# VestaData.reset_index(inplace=True, drop=True)
# VestaData= VestaData[0:limiit]

# AlbertData= shuffle(data[data["speaker"]=="Albert"])
# AlbertData.reset_index(inplace=True, drop=True)
# AlbertData= AlbertData[0:limiit]

# KalevData= shuffle(data[data["speaker"]=="Kalev"])
# KalevData.reset_index(inplace=True, drop=True)
# KalevData= KalevData[0:limiit]

# MariData= shuffle(data[data["speaker"]=="Mari"])
# MariData.reset_index(inplace=True, drop=True)
# MariData= MariData[0:limiit]

# KylliData= shuffle(data[data["speaker"]=="Kylli"])
# KylliData.reset_index(inplace=True, drop=True)
# KylliData= KylliData[0:limiit]

# MeelisData= shuffle(data[data["speaker"]=="Meelis"])
# MeelisData.reset_index(inplace=True, drop=True)
# MeelisData= MeelisData[0:limiit]

# combinedData = VestaData.append([AlbertData,KalevData,MariData,KylliData,MeelisData])

30
